# Modelagem Multinível do Efeito das Emendas PIX

Este notebook implementa a estratégia **multilevel step-up** descrita no TCC.
As etapas são:
1. **Modelo nulo**: estima a variância intra-partidos sem variáveis explanatórias.
2. **Modelo com interceptos aleatórios**: inclui `emendas_pix_per_capita_partido_prefeito_eleito` como efeito fixo, permitindo interceptos diferentes por partido.
3. **Modelo com interceptos e inclinações aleatórios**: além do intercepto, o efeito das emendas por habitante varia entre partidos.
4. **Modelo completo**: adiciona as variáveis *dummy* dos clusters socioeconômicos, controlando perfis municipais.


In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm # estimação de modelos
from scipy import stats # estatística chi2
from statsmodels.iolib.summary2 import summary_col # comparação entre modelos
from scipy.stats import gaussian_kde # inserção de KDEs em gráficos
from matplotlib.gridspec import GridSpec # plotagem de gráficos separados
import time # definição do intervalo de tempo entre gráficos com animação
import imageio # para geração de figura GIF
from tqdm import tqdm # adiciona um indicador de progresso do código


In [ ]:
# Carrega a base unificada com dummies de clusters
base = pd.read_csv('../data/dados_com_clusters.csv')

# Remover 1s da coluna 'porcentual_votos_partido_prefeito_eleito'
base = base[base['porcentagem_votos_validos_2024'] < 1]

# Remover valores extremos da coluna 'emendas_pix_per_capita_partido_prefeito_eleito'
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

base = remove_outliers(base, 'emendas_pix_per_capita_partido_prefeito_eleito')

# Converte as colunas de cluster para inteiro (0/1)
for col in ['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3']:
    base[col] = base[col].fillna(False).astype(int)


In [ ]:
# Modelo nulo
null_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ 1',
    base,
    groups=base['sigla_partido_prefeito_eleito']
)
null_res = null_model.fit()
print(null_res.summary())


In [ ]:
# Modelo com interceptos aleatórios
ri_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
    base,
    groups=base['sigla_partido_prefeito_eleito']
)
ri_res = ri_model.fit()
print(ri_res.summary())


In [ ]:
# Modelo com interceptos e inclinações aleatórios
rs_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    re_formula='1 + emendas_pix_per_capita_partido_prefeito_eleito'
)
rs_res = rs_model.fit()
print(rs_res.summary())


In [ ]:
# Modelo multinível completo com dummies dos clusters
full_model = smf.mixedlm(
    'porcentagem_votos_validos_2024 ~ emendas_pix_per_capita_partido_prefeito_eleito + cluster_0 + cluster_1 + cluster_2 + cluster_3',
    base,
    groups=base['sigla_partido_prefeito_eleito'],
    re_formula='1 + emendas_pix_per_capita_partido_prefeito_eleito'
)
full_res = full_model.fit()
print(full_res.summary())


## Análise dos coeficientes
A seguir resumimos os parâmetros do modelo completo para verificar quais variáveis apresentam efeitos estatisticamente significativos.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
summary_df = pd.DataFrame({
    'coeficiente': full_res.params,
    'erro_padrao': full_res.bse,
    'p_valor': full_res.pvalues
})
summary_df

In [ ]:
from scipy import stats

# exemplo para intercepto aleatório
lr_stat = -2*(ri_res.llf - null_res.llf)
pval = stats.chi2.sf(lr_stat, df=1)
print(f"LR stat={lr_stat:.2f}, p-value={pval:.3f}")

from statsmodels.miscmodels import BetaModel
endog = base['porcentagem_votos_validos_2024']
exog = sm.add_constant(base[['emendas_pix_per_capita', 'idhm','pib_per_capita','densidade']])
beta_mod = BetaModel(endog, exog).fit()
print(beta_mod.summary())


In [ ]:
import statsmodels.formula.api as smf
logit = smf.mixedlm("reeleito ~ emendas_pix_pc + idhm + pib_pc + densidade",
                    base, groups=base["sigla_partido_prefeito_eleito"],
                    family=sm.families.Binomial()).fit()
